In [9]:
import numpy as np
import pandas as pd
import networkx as nx
import pickle
import node2vec

In [5]:
longnames = pd.read_csv("longnames.csv")
longnames_tsn = list(longnames["tsn"])
longnames_completename = list(longnames["completename"])

In [6]:
tsn_to_longnames = {}
longnames_to_tsn = {}
for a,b in zip(longnames_tsn,longnames_completename):
    tsn_to_longnames[str(a)] = b
    longnames_to_tsn[b] = str(a)

In [7]:
hierarchy = pd.read_csv("hierarchy.csv")
hierarchy_list = list(hierarchy["hierarchy_string"])
hierarchy = hierarchy.sort_values(by=['level'],ascending=False)

In [ ]:
lc = pd.read_csv('assessments- least concern.csv',error_bad_lines=True)
ev = pd.read_csv('assessments- everything.csv',error_bad_lines=True)
lc = lc[['scientificName','redlistCategory']]
ev = ev[['scientificName','redlistCategory']]
final_list = pd.concat([lc,ev],axis=0)

In [18]:
species = [x.strip() for x in list(final_list['scientificName'])]
longnames_to_tsn_keys = [x.strip() for x in longnames_to_tsn.keys()]
wanted = list(set(longnames_to_tsn_keys).intersection(set(species)))

with open("wanted.pkl","wb") as file:
    pickle.dump(wanted,file)

In [24]:
wantedNodes = []
for i in wanted:
    wantedNodes.append(longnames_to_tsn[i])

In [25]:
hierarchy_list = list(hierarchy['hierarchy_string'])

In [29]:
with open('filtered_graph.pkl','rb') as file:
    graph = pickle.load(file)

In [30]:
node2vec = node2vec.Node2Vec(graph, dimensions=200, walk_length=16, num_walks=100, workers=1,temp_folder='graph/')


Computing transition probabilities: 100%|██████████| 168154/168154 [01:33<00:00, 1798.45it/s]

Generating walks (CPU: 1): 100%|██████████| 100/100 [5:24:52<00:00, 194.92s/it]


In [31]:
model = node2vec.fit(window=5, min_count=1)

In [32]:
with open('model_pattern.pkl','wb') as file:
    pickle.dump(model,file)

In [38]:
wanted_embedding = {}
for w in wanted:
    try:
        wanted_embedding[w] = model.wv[w]
    except:
        print(w)

Litoria pearsoniana
Oreopsar bolivianus
Litoria piperata
Galagoides zanzibaricus
Litoria nyakalensis
Pipistrellus cadornae
Phrynobatrachus liberiensis
Pleurobema murrayense
Telmatobius zapahuirensis
Capra cylindricornis
Caridina togoensis
Philemon kisserensis
Scinax flavoguttatus
Liolaemus ceii
Vanmanenia pingchowensis
Acanthurus blochii
Litoria manya
Enhydris pakistanica
Alosa fallax
Alburnus timarensis
Larus scoresbii
Fejervarya keralensis
Pipistrellus crassulus
Litoria myola
Hypsiboas semilineatus
Litoria nudidigita
Trichopodus pectoralis
Tepuihyla rimarum
Rhacophorus yinggelingensis
Rhacophorus dulitensis
Rallicula rubra
Hypsiboas lemai
Litoria australis
Hylogomphus abbreviatus
Hypsiboas rufitelus
Geochelone gigantea
Coeligena orina
Ptychohyla erythromma
Eleutherodactylus dixoni
Dermanura cinerea
Gerbillus simoni
Helcogramma striata
Chaetodon lunulatus
Basileuterus hypoleucus
Elaphe moellendorffi
Gymnothorax rueppelliae
Masticophis lateralis
Hypsiboas lanciformis
Lasiopodomys fuscu

In [50]:
len(wanted_embedding)

39054

In [56]:
with open('wanted_embedding.pkl','wb') as file:
     pickle.dump(wanted_embedding,file)

In [ ]:
with open('wanted_df.pkl','rb') as file:
     wanted_df=pickle.load(file)

In [52]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
wanted_df['label'] = le.fit_transform(wanted_df['redlistCategory'])

In [58]:
wanted_df['embedding']= wanted_df.apply(lambda row: wanted_embedding[row['scientificName']],axis=1)

In [62]:
with open('wanted_df.pkl','wb') as file:
     pickle.dump(wanted_df,file)

In [63]:
wanted_df

,scientificName,redlistCategory,embedding
0,Ambloplites cavifrons,Least Concern,"[-0.6297879, -0.26878166, -0.31555513, -0.2876..."
1,Ilyodon whitei,Least Concern,"[-0.57829165, -0.66301274, -0.7258973, -0.7544..."
2,Kuhlia marginata,Least Concern,"[-0.2089876, 0.110810935, -0.33880642, -0.1012..."
3,Lampsilis ornata,Least Concern,"[-0.55644095, -0.52328426, -0.5357609, 0.09064..."
4,Lampsilis splendida,Least Concern,"[-0.575738, -0.45923498, -0.5215285, 0.1471294..."
...,...,...,...
18998,Brachypelma hamorii,Vulnerable,"[-0.575585, -0.47272348, -0.3817137, -0.094923..."
19000,Python bivittatus,Vulnerable,"[-1.9505544, -0.36114147, -1.2164276, 0.240369..."
19001,Phelsuma guentheri,Vulnerable,"[-0.5119155, -0.35328937, -0.4172045, -0.26478..."
19002,Casarea dussumieri,Vulnerable,"[-0.58294284, -0.442629, -0.6558757, 0.1548084..."


In [66]:
set(wanted_df['redlistCategory'])

{'Critically Endangered',
 'Endangered',
 'Extinct',
 'Extinct in the Wild',
 'Least Concern',
 'Lower Risk/conservation dependent',
 'Lower Risk/least concern',
 'Lower Risk/near threatened',
 'Near Threatened',
 'Vulnerable'}

In [73]:
merge_cat = {'Lower Risk/least concern':'Stable',
             'Least Concern':'Stable',
             'Critically Endangered':'Unstable',
             'Lower Risk/conservation dependent':'Unstable',
             'Lower Risk/near threatened':'Unstable',
             'Endangered':'Unstable',
             'Near Threatened':'Unstable',
             'Vulnerable':'Unstable',
             'Extinct':'Unstable',
             'Extinct in the Wild':'Unstable',
            }

In [74]:
wanted_df['redlistCategory'] = wanted_df.apply(
lambda row: merge_cat[row['redlistCategory']],axis=1)

/ilab/users/mj621/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
